In [47]:
import random

def get_sales(store):
    if store <= 4:
        return random.randint(200, 1000) / 1000
    else:
        return random.randint(500, 1500) / 1000


def create_data(output_file):
    weeks = range(1, 53)
    stores = range(1, 11)
    products = ["ZIEMNIAKI", "BURAKI", "KAPUSTA", "MARCHEW"]

    lines = [
        f"{week} {store} {product} {get_sales(store)}"
        for week in weeks
        for store in stores
        for product in products
    ]

    with open(output_file, 'w') as f:
        f.write("param weekly_sales :=\n")
        f.write("\n".join(lines))
        f.write("\n;\n")


In [48]:

create_data("weekly_sales2.dat")

In [49]:
import pandas as pd

# Pomijamy pierwszą linię i ostatnią ze średnikiem
df = pd.read_csv("weekly_sales2.dat",
                 skiprows=1,
                 skipfooter=1,
                 engine="python",
                 delim_whitespace=True,
                 names=["week", "store", "product", "sales"])

/tmp/ipykernel_4239/3307047910.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("weekly_sales2.dat",


In [50]:
df_avg = df.groupby(["store"], as_index=False)["sales"].sum()
df_avg["sales"] = ((df_avg["sales"] / 52) * 2).round(3)
# df_avg

In [51]:
def save_stockroom_capacity(df, filename):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("param stockroom_capacity :=\n")
        for _, row in df.iterrows():
            f.write(f"{int(row['store'])} {row['sales']}\n")
        f.write(";\n")

save_stockroom_capacity(df_avg, "stockroom_capacity2.dat")

In [52]:
producers = {
    "SKIERNIEWICE": (51.966488, 20.149736),
    "KSIĄŻENICE": (52.070476, 20.684361),
    "GÓRA_KALWARIA": (51.980816, 21.216427),
    "OTWOCK": (52.105678, 21.261457),
    "WOŁOMIN": (52.340453, 21.243221),
    "LEGIONOWO": (52.404672, 20.952807)
}

In [53]:
stores = {
    "1": (52.218141, 21.011481),   # Nowowiejska 15/19, 00-665 Warszawa
    "2": (52.383261, 20.926887),   # Akademijna, 05-110 Jabłonna
    "3": (52.275729, 21.082742),   #Wierna 23, 03-890 Warszawa
    "4": (52.290395, 20.929127),   # Nocznickiego, 00-001 Warszawa
    "5": (52.278534, 20.942370),   # Stanisława Przybyszewskiego 32, 01-824 Warszawa
    "6": (52.137360, 20.922216),   # aleja Hrabska 4a, 05-090 Falenty
    "7": (52.288574, 21.108812),   # Zaciszna 20-24, 05-091 Ząbki
    "8": (52.173798, 21.020688),   # Puławska 233-231, Warszawa
    "9": (52.128329, 21.079121),   # Trakt Leśny, 02-797 Warszawa
    "10": (52.317838, 20.961168),  # Myśliborska 85-23, 03-185 Warszawa
}


In [54]:
warehouses = {
    "PRUSZKÓW": (52.170189, 20.817537),
    "PIASECZNO": (52.077273, 21.026131),
    "ZIELONKA": (52.304954, 21.159194),
}

In [55]:
from math import radians, sin, cos, sqrt, atan2

# Funkcja Haversine – oblicza odległość w kilometrach
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # promień Ziemi w km
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)

    a = sin(dphi/2)**2 + cos(phi1) * cos(phi2) * sin(dlambda/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return round(R * c, 2)

In [56]:
# Obliczanie odległości
distance_prod_wh = []
for prod, (plat, plon) in producers.items():
    for wh, (wlat, wlon) in warehouses.items():
        d = haversine(plat, plon, wlat, wlon)
        distance_prod_wh.append((prod, wh, d))

distance_wh_store = []
for wh, (wlat, wlon) in warehouses.items():
    for s, (slat, slon) in stores.items():
        d = haversine(wlat, wlon, slat, slon)
        distance_wh_store.append((wh, s, d))


In [57]:
def save_distance_dat(data, name):
    with open(name, "w", encoding="utf-8") as f:
        f.write(f"param {name.split('.')[0]} :=\n")
        for row in data:
            f.write("  " + " ".join(map(str, row)) + "\n")
        f.write(";\n")

save_distance_dat(distance_prod_wh, "distance_prod_wh.dat")
save_distance_dat(distance_wh_store, "distance_wh_store.dat")